# Example to use safetensor with stable diffusion model

In [1]:
import warnings

from diffusers import StableDiffusionPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from diffusers.utils import logging
from transformers import CLIPImageProcessor

import torch

logging.set_verbosity_error()
warnings.filterwarnings("ignore")

prompt = "Sherlock Holmes"
negative_prompt = ""

safetensor_path = 'models/colorwater-v4.safetensors'

before = torch.cuda.get_device_properties(7).total_memory - torch.cuda.memory_allocated(7)
before /= (1024 ** 2)
#*********************

pipeline = StableDiffusionPipeline.from_single_file(safetensor_path,
                                                    extract_ema=True,
                                                    safety_checker=StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker"),
                                                    feature_extractor=CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32"),
                                                    image_size=512)
# device = "cuda:7" if torch.cuda.is_available() else "cpu"
# pipeline.to(device)

# print(pipeline.device)

# image = pipeline(
#     prompt=prompt,
#     negative_prompt=negative_prompt, 
#     num_inference_steps=150, # sampling steps
#     guidance_scale= 7, # 7, # cfg
#     generator=torch.manual_seed(246372),
# )

# after = torch.cuda.get_device_properties(7).total_memory - torch.cuda.memory_allocated(7)
# after /= (1024 ** 2)

# print(f"usage: {(before-after):.2f} MB")

/local/disk6/chrenx/conda-env/envs/humanai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# device = "cuda:7" if torch.cuda.is_available() else "cpu"
# pipeline.to(device)

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.27.2",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [2]:
img = image.images[0]
img.save("output.png")
if image['nsfw_content_detected'][0]:
    print('yes')
else:
    print('no')
img.size, type(img)

no


((1024, 1024), PIL.Image.Image)

# Debug

In [2]:
a = 7.5e+9
a

7500000000.0

In [10]:
import gradio as gr


def show_text(model_name):
    return gr.update()


demo = gr.Blocks()

with demo:
    gr.Markdown(
        """
    # Show text!
    Start typing below to see the output.
    """
    )
    model_name = gr.Dropdown(label="Select Your Imaging Style", 
                                         choices=[2, 4, 5, 3], value=None)

    model_name.change(fn=show_text, inputs=model_name, outputs=model_name, show_progress = True)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://cf03891c9a0af8d165.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
